In [1]:
!pwd

/data/home/darina.bocharova


In [2]:
personal_info = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [3]:
personal_info

[[23126, 'en', 'Compass - powerful SASS library that makes your life easier'],
 [21617, 'en', 'Preparing for the AP* Computer Science A Exam — Part 2'],
 [16627, 'es', 'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'],
 [11556,
  'es',
  'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'],
 [16704, 'ru', 'Программирование на Lazarus'],
 [13702, 'ru', 'Математическая экономика']]

In [4]:
my_id_cours_list = sorted([personal_info[i][0:2] for i in range(len(personal_info))])
my_id_cours_list

[[11556, 'es'],
 [13702, 'ru'],
 [16627, 'es'],
 [16704, 'ru'],
 [21617, 'en'],
 [23126, 'en']]

In [5]:
import os
import sys

os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 3g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "Spark Dataframe app") 

spark = SparkSession.builder.config(conf=conf).appName("Spark Dataframe app").getOrCreate()

In [7]:
spark

In [8]:
# df = spark.read.json("/labs/slaba02/DO_record_per_line.json")

In [9]:
df = spark.read.json("/labs/slaba02/DO_record_per_line.json").cache()

In [10]:
df

DataFrame[cat: string, desc: string, id: bigint, lang: string, name: string, provider: string]

In [11]:
df.rdd.getNumPartitions()

3

In [12]:
df = df.repartition(6)

In [13]:
df.show(5)

+--------------------+--------------------+-----+----+--------------------+--------+
|                 cat|                desc|   id|lang|                name|provider|
+--------------------+--------------------+-----+----+--------------------+--------+
|3/business_manage...| Unique video ske...|10209|  en|Learn How To Writ...|   Udemy|
|   1/arts_music_film|Explores such ind...|15686|  en|Photoshop CS5 One...|   Lynda|
|  5/computer_science|Learn the basic c...|11858|  en|Human Anatomy and...|   ed2go|
|   1/arts_music_film|How to streamline...|15821|  en|Premiere Pro CS4 ...|   Lynda|
|                    |
Hola a Todos!
En...|10037|  es|Introduccion Visu...|   Udemy|
+--------------------+--------------------+-----+----+--------------------+--------+
only showing top 5 rows



In [14]:
df.count()

28153

In [15]:
cut_df = df.select(["desc", "id", "lang"])
cut_df.show(5)

+--------------------+----+----+
|                desc|  id|lang|
+--------------------+----+----+
|In this class, co...|2930|  en|
|This course will ...|2222|  en|
| Learn Team effec...|9576|  en|
|This course is de...|2530|  en|
| Many commercial-...|5609|  en|
+--------------------+----+----+
only showing top 5 rows



In [16]:
df_en = cut_df.filter(cut_df.lang == "en").select(["id", "desc"])
df_en.show(5)

+-----+--------------------+
|   id|                desc|
+-----+--------------------+
|10593| Learn Oracle SQL...|
|14224|Find out about th...|
|17423|Apple's new iAd f...|
|17843| iOS App Developm...|
|16174|Learn how to buil...|
+-----+--------------------+
only showing top 5 rows



In [17]:
df_en.count()

24553

In [18]:
import pyspark.sql.functions as f

In [19]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [20]:
tokenizer_en = Tokenizer(inputCol="desc", outputCol="words_list")
df_en = tokenizer_en.transform(df_en)
df_en.show(5)

+-----+--------------------+--------------------+
|   id|                desc|          words_list|
+-----+--------------------+--------------------+
|10593| Learn Oracle SQL...|[, learn, oracle,...|
|14224|Find out about th...|[find, out, about...|
|17423|Apple's new iAd f...|[apple's, new, ia...|
|17843| iOS App Developm...|[, ios, app, deve...|
|16174|Learn how to buil...|[learn, how, to, ...|
+-----+--------------------+--------------------+
only showing top 5 rows



In [21]:
# 574, 583, 647, 8010
df_en.filter(df_en.id == 8010).select(["id", "desc", "words_list"]).show(1, vertical=True, truncate=False)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 id         | 8010                                                                                                                                                                                                                                                                                                                                                      
 desc       | NLP這套學問的假設，是我們以NLP視野體驗環境的前提。課程會將NLP的假設前提分為四方面：關於溝通策略的假設前提關於解讀行為的假設前提關於大腦認知的假設前提關於自我提昇的假設前提除此以外，課程亦會介紹「mind to muscle」技巧，學會將一些有啟發性的概念植根於神經系統。這是正式學習NLP的第一課，雖有很多新概念需時消化，但對認識、學習、溫習NLP都非常重要！     
 words_list | [nlp這套學問的假設，是我們以nlp視野體驗環境的前提。課程會將nlp的假設前提分為四方面：關於溝通策略的假設前提關

In [22]:
stop_words_en = StopWordsRemover.loadDefaultStopWords("english")
print(stop_words_en)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [23]:
stw_en = StopWordsRemover(inputCol="words_list", outputCol="filtr_words_list", stopWords=stop_words_en)
df_en = stw_en.transform(df_en)
df_en.show(5)

+-----+--------------------+--------------------+--------------------+
|   id|                desc|          words_list|    filtr_words_list|
+-----+--------------------+--------------------+--------------------+
|10593| Learn Oracle SQL...|[, learn, oracle,...|[, learn, oracle,...|
|14224|Find out about th...|[find, out, about...|[find, surprising...|
|17423|Apple's new iAd f...|[apple's, new, ia...|[apple's, new, ia...|
|17843| iOS App Developm...|[, ios, app, deve...|[, ios, app, deve...|
|16174|Learn how to buil...|[learn, how, to, ...|[learn, build, we...|
+-----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [24]:
hashingTF_en = HashingTF(inputCol="filtr_words_list", outputCol="tf", numFeatures=1000)
tf_en = hashingTF_en.transform(df_en)
df_en = IDF(inputCol="tf", outputCol="features").fit(tf_en).transform(tf_en)
df_en.show(5)

+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  id|                desc|          words_list|    filtr_words_list|                  tf|            features|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|8133|The first MOOC to...|[the, first, mooc...|[first, mooc, tea...|(1000,[1,5,8,18,2...|(1000,[1,5,8,18,2...|
|1629|11.941 and 11.942...|[11.941, and, 11....|[11.941, 11.942, ...|(1000,[19,25,26,3...|(1000,[19,25,26,3...|
|8851| The Course This ...|[, the, course, t...|[, course, second...|(1000,[0,3,19,21,...|(1000,[0,3,19,21,...|
|3937| Stuck in writing...|[, stuck, in, wri...|[, stuck, writing...|(1000,[3,4,6,16,2...|(1000,[3,4,6,16,2...|
|6182| Why is it that s...|[, why, is, it, t...|[, servers, make,...|(1000,[1,6,7,10,1...|(1000,[1,6,7,10,1...|
+----+--------------------+--------------------+--------------------+--------------------+--------------

In [25]:
def cos_simailar_list_en(my_cours_id):
  
    target = df_en.filter(df_en.id == my_cours_id).select(["features"]).take(1)
    target = target[0].__getitem__('features')
  
    cos_simailar_list = []
    for id, features in df_en.select("id", "features").take(df_en.count()):
        cos_simailar = target.dot(features) / (target.norm(2) * features.norm(2))
        # !
        if str(cos_simailar) != 'nan':
            cos_simailar_list.append([id, cos_simailar])
        sorted_cos_simailar_list = sorted(cos_simailar_list, key=lambda x: x[1], reverse=True)
    return sorted_cos_simailar_list[0:11]

In [26]:
total_list_en1 = cos_simailar_list_en(21617)
print(total_list_en1)
total_list_en1_id = []
for i in range(10):
    total_list_en1_id.append(total_list_en1[i+1][0])
print(total_list_en1_id)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


[[21617, 0.9999999999999998], [21609, 0.9879770695632255], [21608, 0.5121478668212629], [21616, 0.5099089184033334], [21492, 0.4150819256540292], [21624, 0.40131357525456773], [21630, 0.3958597768210409], [21628, 0.39558891364916526], [21623, 0.39467405336620726], [21474, 0.35134058978367644], [16162, 0.34287712829023914]]
[21609, 21608, 21616, 21492, 21624, 21630, 21628, 21623, 21474, 16162]


In [27]:
total_list_en2 = cos_simailar_list_en(23126)
print(total_list_en2)
total_list_en2_id = []
for i in range(10):
    total_list_en2_id.append(total_list_en2[i+1][0])
print(total_list_en2_id)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


[[23126, 1.0], [10764, 0.5086536515468937], [13782, 0.4829889650994544], [13727, 0.4827985373316882], [5114, 0.4633097723776149], [24419, 0.45925418609589813], [9949, 0.44787706610113887], [13665, 0.42581447134167366], [20638, 0.4205004686919404], [2724, 0.4090014421469624], [23478, 0.39602540765777966]]
[10764, 13782, 13727, 5114, 24419, 9949, 13665, 20638, 2724, 23478]


In [28]:
df_es = cut_df.filter(cut_df.lang == "es")
df_es.show(5)

+--------------------+-----+----+
|                desc|   id|lang|
+--------------------+-----+----+
| Aprenderás nueva...|12801|  es|
| En este curso va...|20070|  es|
|
Category:
Business |10491|  es|
| Aprende las técn...|10381|  es|
|Este curso dotará...|18285|  es|
+--------------------+-----+----+
only showing top 5 rows



In [29]:
df_es.count()

1374

In [30]:
tokenizer_es = Tokenizer(inputCol="desc", outputCol="words_list")
df_es = tokenizer_en.transform(df_es)
df_es.show(5)

+--------------------+----+----+--------------------+
|                desc|  id|lang|          words_list|
+--------------------+----+----+--------------------+
| Aprende a realiz...|7121|  es|[, aprende, a, re...|
| Curso de Técnica...|9421|  es|[, curso, de, téc...|
| The Course Este ...|3660|  es|[, the, course, e...|
| De la Identidad ...|5492|  es|[, de, la, identi...|
| Aprende cómo ens...|9048|  es|[, aprende, cómo,...|
+--------------------+----+----+--------------------+
only showing top 5 rows



In [31]:
hashingTF_es = HashingTF(inputCol="words_list", outputCol="tf", numFeatures=1000)
tf_es = hashingTF_es.transform(df_es)
df_es = IDF(inputCol="tf", outputCol="features").fit(tf_es).transform(tf_es)
df_es.show(5)

+--------------------+-----+----+--------------------+--------------------+--------------------+
|                desc|   id|lang|          words_list|                  tf|            features|
+--------------------+-----+----+--------------------+--------------------+--------------------+
| Aprenderás nueva...|12801|  es|[, aprenderás, nu...|(1000,[3,14,21,34...|(1000,[3,14,21,34...|
| En este curso va...|20070|  es|[, en, este, curs...|(1000,[3,7,17,18,...|(1000,[3,7,17,18,...|
|
Category:
Business |10491|  es|[, category:, bus...|(1000,[372,374,52...|(1000,[372,374,52...|
| Aprende las técn...|10381|  es|[, aprende, las, ...|(1000,[0,1,3,14,1...|(1000,[0,1,3,14,1...|
|Este curso dotará...|18285|  es|[este, curso, dot...|(1000,[1,3,7,14,1...|(1000,[1,3,7,14,1...|
+--------------------+-----+----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [32]:
def cos_simailar_list_es(my_cours_id):
  
  target = df_es.filter(df_es.id == my_cours_id).select(["features"]).take(1)
  target = target[0].__getitem__('features')
  
  cos_simailar_list = []
  for id, features in df_es.select("id", "features").take(df_es.count()):
    cos_simailar = target.dot(features) / (target.norm(2) * features.norm(2))
    if str(cos_simailar) != 'nan':
        cos_simailar_list.append([id, cos_simailar])
    sorted_cos_simailar_list = sorted(cos_simailar_list, key=lambda x: x[1], reverse=True)
  return sorted_cos_simailar_list[0:11]

In [33]:
total_list_es1 = cos_simailar_list_es(11556)
print(total_list_es1)
total_list_es1_id = []
for i in range(10):
  total_list_es1_id.append(total_list_es1[i+1][0])
print(total_list_es1_id)

[[11556, 1.0], [16488, 0.40925983599682725], [22710, 0.32061954484074584], [468, 0.3081773509475058], [10384, 0.30176738830370875], [19330, 0.2896889536301552], [11605, 0.2820844232758169], [13461, 0.2651682506782593], [10447, 0.2594332624681442], [11568, 0.24387608198892807], [11564, 0.21751557562489288]]
[16488, 22710, 468, 10384, 19330, 11605, 13461, 10447, 11568, 11564]


In [34]:
total_list_es2 = cos_simailar_list_es(16627)
print(total_list_es2)
total_list_es2_id = []
for i in range(10):
  total_list_es2_id.append(total_list_es2[i+1][0])
print(total_list_es2_id)

[[16627, 1.0000000000000002], [11431, 0.4962725387738898], [12660, 0.3600395508529288], [12247, 0.35905200419482813], [17964, 0.33640299384420097], [5687, 0.3336725668346817], [9598, 0.3292829914868768], [25010, 0.32705885254453787], [16694, 0.31186150164549387], [17961, 0.3068127903204994], [11575, 0.29121485119152507]]
[11431, 12660, 12247, 17964, 5687, 9598, 25010, 16694, 17961, 11575]


In [35]:
df_ru = cut_df.filter(cut_df.lang == "ru")
df_ru.show(5)

+--------------------+-----+----+
|                desc|   id|lang|
+--------------------+-----+----+
|Курс является вве...|13698|  ru|
|Курс посвящен исп...|20357|  ru|
|Онлайн-интенсив п...|17075|  ru|
|Практический курс...|18279|  ru|
|В курсе излагаютс...|13696|  ru|
+--------------------+-----+----+
only showing top 5 rows



In [36]:
df_ru.count()

1231

In [37]:
tokenizer_ru = Tokenizer(inputCol="desc", outputCol="words_list")
df_ru = tokenizer_ru.transform(df_ru)
df_ru.show(5)

+--------------------+----+----+--------------------+
|                desc|  id|lang|          words_list|
+--------------------+----+----+--------------------+
|Курс рассчитан на...|1063|  ru|[курс, рассчитан,...|
|Курс посвящен изу...|1020|  ru|[курс, посвящен, ...|
|В курсе изучаются...|1151|  ru|[в, курсе, изучаю...|
|Технический трени...|1395|  ru|[технический, тре...|
|В этом курсе пере...|1356|  ru|[в, этом, курсе, ...|
+--------------------+----+----+--------------------+
only showing top 5 rows



In [38]:
stop_words_ru = StopWordsRemover.loadDefaultStopWords("russian")
print(stop_words_ru)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [39]:
stw_ru = StopWordsRemover(inputCol="words_list", outputCol="filtr_words_list", stopWords=stop_words_ru)
df_ru = stw_ru.transform(df_ru)
df_ru.show(5)

+--------------------+----+----+--------------------+--------------------+
|                desc|  id|lang|          words_list|    filtr_words_list|
+--------------------+----+----+--------------------+--------------------+
|Курс рассчитан на...|1063|  ru|[курс, рассчитан,...|[курс, рассчитан,...|
|Курс посвящен изу...|1020|  ru|[курс, посвящен, ...|[курс, посвящен, ...|
|В курсе изучаются...|1151|  ru|[в, курсе, изучаю...|[курсе, изучаются...|
|Технический трени...|1395|  ru|[технический, тре...|[технический, тре...|
|В этом курсе пере...|1356|  ru|[в, этом, курсе, ...|[курсе, перемести...|
+--------------------+----+----+--------------------+--------------------+
only showing top 5 rows



In [40]:
hashingTF_ru = HashingTF(inputCol="filtr_words_list", outputCol="tf", numFeatures=1000)
tf_ru = hashingTF_ru.transform(df_ru)
df_ru = IDF(inputCol="tf", outputCol="features").fit(tf_ru).transform(tf_ru)
df_ru.show(5)

+--------------------+-----+----+--------------------+--------------------+--------------------+--------------------+
|                desc|   id|lang|          words_list|    filtr_words_list|                  tf|            features|
+--------------------+-----+----+--------------------+--------------------+--------------------+--------------------+
|Курс является вве...|13698|  ru|[курс, является, ...|[курс, является, ...|(1000,[0,34,181,2...|(1000,[0,34,181,2...|
|Курс посвящен исп...|20357|  ru|[курс, посвящен, ...|[курс, посвящен, ...|(1000,[3,175,214,...|(1000,[3,175,214,...|
|Онлайн-интенсив п...|17075|  ru|[онлайн-интенсив,...|[онлайн-интенсив,...|(1000,[18,27,34,5...|(1000,[18,27,34,5...|
|Практический курс...|18279|  ru|[практический, ку...|[практический, ку...|(1000,[13,29,34,4...|(1000,[13,29,34,4...|
|В курсе излагаютс...|13696|  ru|[в, курсе, излага...|[курсе, излагаютс...|(1000,[17,32,44,9...|(1000,[17,32,44,9...|
+--------------------+-----+----+--------------------+--

In [41]:
def cos_simailar_list_ru(my_cours_id):
  
  target = df_ru.filter(df_ru.id == my_cours_id).select(["features"]).take(1)
  target = target[0].__getitem__('features')
  
  cos_simailar_list = []
  for id, features in df_ru.select("id", "features").take(df_ru.count()):
    cos_simailar = target.dot(features) / (target.norm(2) * features.norm(2))
    if str(cos_simailar) != 'nan':
        cos_simailar_list.append([id, cos_simailar])
    sorted_cos_simailar_list = sorted(cos_simailar_list, key=lambda x: x[1], reverse=True)
  return sorted_cos_simailar_list[0:11]

In [42]:
total_list_ru1 = cos_simailar_list_ru(13702)
print(total_list_ru1)
total_list_ru1_id = []
for i in range(10):
  total_list_ru1_id.append(total_list_ru1[i+1][0])
print(total_list_ru1_id)

[[13702, 1.0000000000000002], [864, 1.0000000000000002], [17075, 0.2734349522351883], [919, 0.1912376381558141], [28074, 0.1809981148998331], [13057, 0.17405058778615892], [895, 0.15587750767847933], [1383, 0.1557139972947489], [1052, 0.14260108365282806], [1125, 0.14242270380898678], [8300, 0.13982046314295313]]
[864, 17075, 919, 28074, 13057, 895, 1383, 1052, 1125, 8300]


In [43]:
total_list_ru2 = cos_simailar_list_ru(16704)
print(total_list_ru2)
total_list_ru2_id = []
for i in range(10):
  total_list_ru2_id.append(total_list_ru2[i+1][0])
print(total_list_ru2_id)

[[16704, 1.0], [1250, 0.2613547630305457], [1131, 0.24244690401997568], [1164, 0.2392302084437246], [1228, 0.21866946329918455], [1011, 0.19977351331683688], [25916, 0.19138615349740062], [1257, 0.18929692993551903], [6928, 0.1875958859087456], [1151, 0.17534061921744115], [1361, 0.17336934061580564]]
[1250, 1131, 1164, 1228, 1011, 25916, 1257, 6928, 1151, 1361]


In [44]:
import json
data = {}
# es1
data['11556'] = total_list_es1_id
# ru1
data['13702'] = total_list_ru1_id
# es2
data['16627'] = total_list_es2_id
# ru2
data['16704'] = total_list_ru2_id
# en1
data['21617'] = total_list_en1_id
# en2
data['23126'] = total_list_en2_id
with open('lab02.json', 'w') as outfile:
    json.dump(data, outfile)

In [45]:
spark.stop()

In [46]:
spark

AttributeError: 'NoneType' object has no attribute 'sc'